In [6]:
import sqlite3
import pandas as pd

# Connect to SQLite (this will create bank_reviews.db if it doesn't exist)
conn = sqlite3.connect("../data/bank_reviews.db")
cursor = conn.cursor()
df = pd.read_csv("../data/processed/bank_reviews_with_themes.csv")


# Create Banks Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS banks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);
""")

# Create Reviews Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review TEXT,
    rating INTEGER,
    date TEXT,
    bank_id INTEGER,
    source TEXT,
    label TEXT,
    score REAL,
    keywords TEXT,
    themes TEXT,
    FOREIGN KEY(bank_id) REFERENCES banks(id)
);
""")

conn.commit()


In [7]:
# Example of your df.head():
# review, rating, date, bank, source, label, score, keywords, identified_themes

# Insert unique banks into banks table
bank_names = df["bank"].unique()
for name in bank_names:
    cursor.execute("INSERT OR IGNORE INTO banks (name) VALUES (?)", (name,))
conn.commit()

# Prepare and insert into reviews
for _, row in df.iterrows():
    cursor.execute("SELECT id FROM banks WHERE name = ?", (row["bank"],))
    bank_id = cursor.fetchone()[0]

    cursor.execute("""
    INSERT INTO reviews (
        review, rating, date, bank_id, source, label, score, keywords, themes
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row["review"], row["rating"], row["date"], bank_id, row["source"], row["label"],
        row["score"], ",".join(row["keywords"]), ",".join(row["identified_themes"])
    ))
conn.commit()


In [ ]:
# Count entries
cursor.execute("SELECT COUNT(*) FROM reviews")
print("Number of reviews inserted:", cursor.fetchone()[0])
